In [4]:
import numpy as np
import pandas as pd
import lightgbm
from sklearn.model_selection import train_test_split

In [5]:
train_path = '../DATA/train_normalized_4labels.csv'
test_path = '../DATA/test_normalized_4labels.csv'

In [6]:
data = pd.read_csv(train_path,index_col=0)

/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [7]:
X = data.drop(columns = ['EMPLOYER_NAME','CASE_STATUS']).values
y = data['CASE_STATUS'].values
T = {'CERTIFIED':0, 'CERTIFIED-WITHDRAWN':1, 'DENIED':2, 'WITHDRAWN':3}
y = list(map(lambda x:T[x],y))

In [8]:
X_train, X_valid, y_train, y_valid = train_test_split(X,y,test_size=0.05, random_state=42)

In [6]:
train_data = lightgbm.Dataset(X_train, label=y_train)
valid_data = lightgbm.Dataset(X_valid, label=y_valid)

In [7]:
parameters = {
    'objective': 'ova',
    'is_unbalance': 'true',
    'boosting': 'gbdt',
    'num_leaves': 31,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 20,
    'learning_rate': 0.05,
    'verbose': 0,
    'num_class':4,
    'deterministic':'true'
}

In [8]:
model = lightgbm.train(parameters,
                       train_data,
                       valid_sets=valid_data,
                       num_boost_round=5000,
                       early_stopping_rounds=100)

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.018364 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[1]	valid_0's multi_logloss: 0.384233
Training until validation scores don't improve for 100 rounds
[2]	valid_0's multi_logloss: 0.338526
[3]	valid_0's multi_logloss: 0.323939
[4]	valid_0's multi_logloss: 0.316006
[5]	valid_0's multi_logloss: 0.299488
[6]	valid_0's multi_logloss: 0.281675
[7]	valid_0's multi_logloss: 0.267191
[8]	valid_0's multi_logloss: 0.254878
[9]	valid_0's multi_logloss: 0.252778
[10]	valid_0's multi_logloss: 0.242857
[11]	valid_0's multi_logloss: 0.243258
[12]	valid_0's multi_logloss: 0.234351
[13]	valid_0's multi_logloss: 0.236009
[14]	valid_0's multi_logloss: 0.236622
[15]	valid_0's multi_logloss: 0.235923
[16]	valid_0's multi_logloss: 0.228179
[17]	valid_0's multi_logloss: 0.228288
[18]	valid_0's multi_logloss: 0.228439
[19]	valid_0's m

[202]	valid_0's multi_logloss: 0.0871565
[203]	valid_0's multi_logloss: 0.0871853
[204]	valid_0's multi_logloss: 0.0871476
[205]	valid_0's multi_logloss: 0.0871761
[206]	valid_0's multi_logloss: 0.0872068
[207]	valid_0's multi_logloss: 0.0871761
[208]	valid_0's multi_logloss: 0.0871426
[209]	valid_0's multi_logloss: 0.087164
[210]	valid_0's multi_logloss: 0.0871834
[211]	valid_0's multi_logloss: 0.0872062
[212]	valid_0's multi_logloss: 0.0871611
[213]	valid_0's multi_logloss: 0.0871705
[214]	valid_0's multi_logloss: 0.08718
[215]	valid_0's multi_logloss: 0.0871334
[216]	valid_0's multi_logloss: 0.0871515
[217]	valid_0's multi_logloss: 0.0871072
[218]	valid_0's multi_logloss: 0.0870629
[219]	valid_0's multi_logloss: 0.0870736
[220]	valid_0's multi_logloss: 0.0870883
[221]	valid_0's multi_logloss: 0.0870495
[222]	valid_0's multi_logloss: 0.0870504
[223]	valid_0's multi_logloss: 0.0870059
[224]	valid_0's multi_logloss: 0.0869735
[225]	valid_0's multi_logloss: 0.0869704
[226]	valid_0's mul

[403]	valid_0's multi_logloss: 0.0854854
[404]	valid_0's multi_logloss: 0.0854506
[405]	valid_0's multi_logloss: 0.0854301
[406]	valid_0's multi_logloss: 0.0853989
[407]	valid_0's multi_logloss: 0.0853701
[408]	valid_0's multi_logloss: 0.0853388
[409]	valid_0's multi_logloss: 0.0853178
[410]	valid_0's multi_logloss: 0.0852964
[411]	valid_0's multi_logloss: 0.0852755
[412]	valid_0's multi_logloss: 0.0852538
[413]	valid_0's multi_logloss: 0.0852281
[414]	valid_0's multi_logloss: 0.0852102
[415]	valid_0's multi_logloss: 0.0851849
[416]	valid_0's multi_logloss: 0.0853253
[417]	valid_0's multi_logloss: 0.0853026
[418]	valid_0's multi_logloss: 0.0852329
[419]	valid_0's multi_logloss: 0.0852099
[420]	valid_0's multi_logloss: 0.0851933
[421]	valid_0's multi_logloss: 0.08519
[422]	valid_0's multi_logloss: 0.0851927
[423]	valid_0's multi_logloss: 0.085201
[424]	valid_0's multi_logloss: 0.085205
[425]	valid_0's multi_logloss: 0.0852108
[426]	valid_0's multi_logloss: 0.0852098
[427]	valid_0's mult

[606]	valid_0's multi_logloss: 0.0845538
[607]	valid_0's multi_logloss: 0.0845415
[608]	valid_0's multi_logloss: 0.0845404
[609]	valid_0's multi_logloss: 0.0845356
[610]	valid_0's multi_logloss: 0.0845215
[611]	valid_0's multi_logloss: 0.0845164
[612]	valid_0's multi_logloss: 0.0845097
[613]	valid_0's multi_logloss: 0.0845037
[614]	valid_0's multi_logloss: 0.0845003
[615]	valid_0's multi_logloss: 0.0844939
[616]	valid_0's multi_logloss: 0.0844905
[617]	valid_0's multi_logloss: 0.084483
[618]	valid_0's multi_logloss: 0.0844807
[619]	valid_0's multi_logloss: 0.0844784
[620]	valid_0's multi_logloss: 0.0844745
[621]	valid_0's multi_logloss: 0.084469
[622]	valid_0's multi_logloss: 0.0844579
[623]	valid_0's multi_logloss: 0.0844524
[624]	valid_0's multi_logloss: 0.0844495
[625]	valid_0's multi_logloss: 0.084448
[626]	valid_0's multi_logloss: 0.0844433
[627]	valid_0's multi_logloss: 0.0844376
[628]	valid_0's multi_logloss: 0.0844338
[629]	valid_0's multi_logloss: 0.0844141
[630]	valid_0's mul

[808]	valid_0's multi_logloss: 0.0837071
[809]	valid_0's multi_logloss: 0.0836811
[810]	valid_0's multi_logloss: 0.0836555
[811]	valid_0's multi_logloss: 0.0836389
[812]	valid_0's multi_logloss: 0.0836106
[813]	valid_0's multi_logloss: 0.0835909
[814]	valid_0's multi_logloss: 0.0835638
[815]	valid_0's multi_logloss: 0.0835435
[816]	valid_0's multi_logloss: 0.0835215
[817]	valid_0's multi_logloss: 0.0835051
[818]	valid_0's multi_logloss: 0.0834886
[819]	valid_0's multi_logloss: 0.0834734
[820]	valid_0's multi_logloss: 0.0834507
[821]	valid_0's multi_logloss: 0.0834741
[822]	valid_0's multi_logloss: 0.0835015
[823]	valid_0's multi_logloss: 0.0835246
[824]	valid_0's multi_logloss: 0.0835514
[825]	valid_0's multi_logloss: 0.0835733
[826]	valid_0's multi_logloss: 0.0835986
[827]	valid_0's multi_logloss: 0.0836202
[828]	valid_0's multi_logloss: 0.0836396
[829]	valid_0's multi_logloss: 0.0836573
[830]	valid_0's multi_logloss: 0.0836726
[831]	valid_0's multi_logloss: 0.0836852
[832]	valid_0's 

In [9]:
def calculate_acc(y_test,y_pred):
    labels = [[0,0],[0,0],[0,0],[0,0]] #truth, correct
    for y,yp in zip(y_test,y_pred):
        labels[y][0] += 1
        if y == yp:
            labels[y][1] += 1
    for stat in range(4):
        labels[stat] = labels[stat][1]/labels[stat][0]
    return labels

In [10]:
y_pred = model.predict(X_train).argmax(axis=1)
print("Accuracy on training set:\n",list(zip(['CERTIFIED', 'CERTIFIED-WITHDRAWN', 'DENIED', 'WITHDRAWN'],
               calculate_acc(y_train,y_pred))))

Accuracy on training set:
 [('CERTIFIED', 0.960469667867315), ('CERTIFIED-WITHDRAWN', 0.9698078839742929), ('DENIED', 0.8084626833935785), ('WITHDRAWN', 0.8286909420359697)]


In [11]:
y_pred = model.predict(X_valid).argmax(axis=1)
print("Accuracy on validation set:\n",list(zip(['CERTIFIED', 'CERTIFIED-WITHDRAWN', 'DENIED', 'WITHDRAWN'],
               calculate_acc(y_valid,y_pred))))

Accuracy on validation set:
 [('CERTIFIED', 0.9603258048631167), ('CERTIFIED-WITHDRAWN', 0.9667420083952212), ('DENIED', 0.772911597729116), ('WITHDRAWN', 0.8401500938086304)]


In [12]:
data = pd.read_csv(test_path,index_col=0)
X_test = data.drop(columns = ['EMPLOYER_NAME','CASE_STATUS']).values
y_test = data['CASE_STATUS'].values
del(data)
T = {'CERTIFIED':0, 'CERTIFIED-WITHDRAWN':1, 'DENIED':2, 'WITHDRAWN':3}
y_test = list(map(lambda x:T[x],y_test))

In [13]:
y_pred = model.predict(X_test).argmax(axis=1)
print("Accuracy on test set:\n",list(zip(['CERTIFIED', 'CERTIFIED-WITHDRAWN', 'DENIED', 'WITHDRAWN'],
               calculate_acc(y_test,y_pred))))

Accuracy on test set:
 [('CERTIFIED', 0.960316141704774), ('CERTIFIED-WITHDRAWN', 0.9650138003235937), ('DENIED', 0.7749387139355082), ('WITHDRAWN', 0.821475935828877)]
